# NAACL 2018 Shared Task - Metaphor Detection

Add description

## Prerequisites 

- Facebook FastText Embeddings for English
- https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md

## Preflight Checks

- Installed requirements.txt
- (optional) Download vuamc.zip http://ota.ahds.ac.uk/headers/2541.xml

https://github.com/EducationalTestingService/metaphor/tree/master/NAACL-FLP-shared-task

In [ ]:
# Check for dependencies

import os
import utils
import corpus
import collections
# import numpy
# import keras

In [ ]:
# Check for VUAMC CSV files and generate if necessary

if not os.path.exists('source/vuamc_corpus_test.csv') and not os.path.exists('source/vuamc_corpus_train.csv'):
    print('VUAMC training and test data not found. Generating...')
    # utils.download_vuamc_xml()
    # utils.generate_vuamc_csv()

In [ ]:
# Load Train Corpus from CSV
c_train = corpus.VUAMC('source/vuamc_corpus_train.csv', 'source/verb_tokens_train_gold_labels.csv')
c_train.validate_corpus()
print('Loaded and validated training corpus')

In [ ]:
# Shows that we got imbalanced classes in the training data
number_of_all_labels = len( c_train.label_list)
count_of_label_classes = collections.Counter( c_train.label_list)

percentage_of_non_metaphor_tokens = round(count_of_label_classes[0] / number_of_all_labels * 100)
percentage_of_metaphor_tokens = round(count_of_label_classes[1] / number_of_all_labels * 100)
ratio = utils.simplify_ratio(percentage_of_non_metaphor_tokens, percentage_of_metaphor_tokens)
assert(percentage_of_non_metaphor_tokens + percentage_of_metaphor_tokens == 100)

print('Percentage of metaphor tokens: {}'.format(percentage_of_metaphor_tokens))
print('Percentage of non-metaphor tokens: {}'.format(percentage_of_non_metaphor_tokens))
print('Ratio: {}:{}'.format(ratio[0], ratio[1]))